In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [3]:
pip install youtube-transcript-api

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from youtube_transcript_api import YouTubeTranscriptApi

Transscript = YouTubeTranscriptApi.get_transcript('93gcZEtmL7s&list=PLd9hKAUC3AZuo7is-aN45pmfDwJHOqKAj&index=2')

In [5]:
Transscript

[{'text': 'okay this is the second lecture of the',
  'start': 0.03,
  'duration': 3.89},
 {'text': 'condensed matter course welcome back',
  'start': 2.04,
  'duration': 4.11},
 {'text': 'three quick things before we start a',
  'start': 3.92,
  'duration': 3.64},
 {'text': 'bunch of people after the first lecture',
  'start': 6.15,
  'duration': 3.27},
 {'text': "asked me what's the title of the book so",
  'start': 7.56,
  'duration': 2.97},
 {'text': 'they can find it in the bookstore',
  'start': 9.42,
  'duration': 2.639},
 {'text': "it's the Oxford solid state basics now",
  'start': 10.53,
  'duration': 4.559},
 {'text': 'you can find it the second thing I want',
  'start': 12.059,
  'duration': 5.011},
 {'text': 'to encourage you to use the message',
  'start': 15.089,
  'duration': 4.141},
 {'text': "board no one's used it yet someone be",
  'start': 17.07,
  'duration': 3.81},
 {'text': 'brave post the first message once you',
  'start': 19.23,
  'duration': 2.94},
 {'text':

In [6]:
formatted_data = [f"{entry['start']:.2f}: {entry['text']}" for entry in Transscript]
formatted_text = "\n ".join(formatted_data)
formatted_text

"0.03: okay this is the second lecture of the\n 2.04: condensed matter course welcome back\n 3.92: three quick things before we start a\n 6.15: bunch of people after the first lecture\n 7.56: asked me what's the title of the book so\n 9.42: they can find it in the bookstore\n 10.53: it's the Oxford solid state basics now\n 12.06: you can find it the second thing I want\n 15.09: to encourage you to use the message\n 17.07: board no one's used it yet someone be\n 19.23: brave post the first message once you\n 20.88: get going I promise you'll find it\n 22.17: useful the third thing I didn't\n 23.79: emphasize this enough last time if\n 25.38: there's any problems with the course any\n 27.33: sort of error typos and the homework\n 29.43: sets typos in the book anything I say if\n 31.80: it seems wrong in lecture please come to\n 33.57: me immediately and I can issue\n 35.25: corrections and therefore not get\n 36.96: everyone is you know it confused as much\n 38.58: as we can so just you 

In [7]:
prompt = f"""
Your task is to generate a short summary of a video
using its subtitles. 

You will be provided with text delimited by triple quotes. 
If it contains video subtitles, \ 
generate a short summary of a video:

If the text does not contain video subtitles, \ 
then simply write \"No subtitles provided.\"

\"\"\"{formatted_text}\"\"\"
"""

In [8]:
response = get_completion(prompt)
print(response)

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 15410 tokens. Please reduce the length of the messages.

In [9]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [31]:
import tiktoken

In [29]:
def read_str_create_chunks(text, chunkSize, overlap):
    tt_encoding = tiktoken.get_encoding("cl100k_base")
    
    tokens = tt_encoding.encode(text)
    total_tokens = len(tokens)
    print(total_tokens)
    chunks = []
    for i in range(0, total_tokens, chunkSize-overlap):
        chunk = tokens[i:i+chunkSize]
        chunks.append(chunk)
    return chunks

In [32]:
chunks = read_str_create_chunks(prompt, 4000, 200)

15403


In [33]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

final_responce = []
tt_encoding = tiktoken.get_encoding("cl100k_base")
for index, chunk in enumerate(chunks):
    response = get_completion(tt_encoding.decode(chunk))
    final_responce.append(response)

'In this lecture, the speaker discusses the behavior of waves in solids and the properties of metals. They mention that at very small wavelengths or high wave vectors, sound cannot be accurately described. They also mention that the Debye frequency is not exact for any material, and that metals have different properties compared to other materials. The speaker then introduces the Drude theory of metals, which is a classical kinetic theory that treats electrons in metals as a gas. They discuss'

In [ ]:
prompt = f"""
Your task is to analyze the above transcript 
and generate a list of main topics discussed in 
the video and at what time the discussion takes place (i.e., timestamps).

Please analyze the above transcript and provide a list of the main topics 
discussed in the video along with their corresponding timestamps.

If the text does not contain video subtitles, simply write "No subtitles provided."

{formatted_text}
"""